In [2]:
import cv2
import numpy as np
from pathlib import Path
from skimage.morphology import thin
import matplotlib.pyplot as plt
import logging


class NetworkImageProcessor:
    """网络图像处理类，支持批量处理"""

    def __init__(self):
        self.supported_formats = {'.jpg', '.jpeg', '.png', '.tif', '.tiff', '.bmp', '.gif'}
        self.setup_logging()

    def setup_logging(self):
        """设置日志记录"""
        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s',
            handlers=[
                logging.FileHandler('image_processing.log', encoding='utf-8'),
                logging.StreamHandler()
            ]
        )
        self.logger = logging.getLogger(__name__)

    def get_image_files(self, input_folder):
        """获取文件夹中的所有图像文件"""
        image_files = []

        for file_path in input_folder.rglob('*'):
            if file_path.suffix.lower() in self.supported_formats:
                image_files.append(file_path)

        self.logger.info(f"找到 {len(image_files)} 个图像文件")
        return image_files

    def create_output_structure(self, input_folder):
        """创建输出文件夹结构"""
        output_folder = input_folder / "processed_results"
        subfolders = {
            'processed': output_folder / "processed_images",
            'skeleton': output_folder / "skeleton_images",
            'junction': output_folder / "junction_visualization",
            'reports': output_folder / "processing_reports"
        }

        for folder in subfolders.values():
            folder.mkdir(parents=True, exist_ok=True)

        return subfolders

    def find_junction_points(self, skeleton):
        """找到骨架图像中的分叉点"""
        skeleton_binary = (skeleton > 0).astype(np.uint8)

        kernel = np.array([[1, 1, 1],
                           [1, 0, 1],
                           [1, 1, 1]], dtype=np.uint8)

        neighbor_count = cv2.filter2D(skeleton_binary, -1, kernel)

        junction_mask = (skeleton_binary > 0) & (neighbor_count >= 3)

        return junction_mask

    def remove_junction_points(self, skeleton, junction_mask, removal_radius=1):
        """删除分叉点及其周围像素"""
        result = skeleton.copy().astype(np.float64)

        struct_elem = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,
                                                (removal_radius * 2 + 1, removal_radius * 2 + 1))

        removal_area = cv2.dilate(junction_mask.astype(np.uint8), struct_elem)

        result[removal_area > 0] = 0

        return result

    def process_single_image(self, image_path, output_folders):
        """处理单张图像"""
        try:
            img = cv2.imread(str(image_path), cv2.IMREAD_GRAYSCALE)
            if img is None:
                raise ValueError(f"无法读取图像: {image_path}")

            _, binary = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY_INV)

            skeleton = thin(binary / 255).astype(np.uint8) * 255

            skeleton_normalized = skeleton / 255

            junction_mask = self.find_junction_points(skeleton_normalized)

            processed = self.remove_junction_points(skeleton_normalized, junction_mask)
            processed = (processed * 255).astype(np.uint8)

            kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
            processed = cv2.morphologyEx(processed, cv2.MORPH_CLOSE, kernel)

            base_name = image_path.stem

            processed_path = output_folders['processed'] / f"{base_name}_processed.png"
            cv2.imwrite(str(processed_path), 255 - processed)

            skeleton_path = output_folders['skeleton'] / f"{base_name}_skeleton.png"
            cv2.imwrite(str(skeleton_path), skeleton)

            self.create_visualization(
                img, skeleton, junction_mask, processed,
                output_folders['junction'], base_name
            )

            junction_count = np.sum(junction_mask)

            processing_info = {
                'filename': image_path.name,
                'original_size': img.shape,
                'junction_count': junction_count,
                'success': True,
                'error_message': None
            }

            self.logger.info(f"成功处理: {image_path.name}, 找到 {junction_count} 个分叉点")

            return processing_info

        except Exception as e:
            error_msg = f"处理 {image_path.name} 时发生错误: {str(e)}"
            self.logger.error(error_msg)

            return {
                'filename': image_path.name,
                'success': False,
                'error_message': str(e)
            }

    def create_visualization(self, original, skeleton, junctions, processed, output_folder, base_name):
        """创建可视化图像"""
        plt.figure(figsize=(15, 5))

        plt.subplot(1, 4, 1)
        plt.imshow(original, cmap='gray')
        plt.title('原始图像')
        plt.axis('off')

        plt.subplot(1, 4, 2)
        plt.imshow(skeleton, cmap='gray')
        plt.title('骨架提取')
        plt.axis('off')

        plt.subplot(1, 4, 3)
        junction_points = np.zeros_like(original)
        junction_points[junctions] = 255
        plt.imshow(skeleton, cmap='gray')
        plt.imshow(junction_points, cmap='Reds', alpha=0.7)
        plt.title(f'分叉点标记 (共{np.sum(junctions)}个)')
        plt.axis('off')

        plt.subplot(1, 4, 4)
        plt.imshow(255 - processed, cmap='gray')
        plt.title('删除分叉点后')
        plt.axis('off')

        plt.tight_layout()

        visualization_path = output_folder / f"{base_name}_visualization.png"
        plt.savefig(str(visualization_path), dpi=150, bbox_inches='tight')
        plt.close()

    def generate_report(self, processing_results, output_folder):
        """生成处理报告"""
        report_path = output_folder / "processing_report.txt"

        successful = [r for r in processing_results if r['success']]
        failed = [r for r in processing_results if not r['success']]

        with open(report_path, 'w', encoding='utf-8') as f:
            f.write("图像处理报告\n")
            f.write("=" * 50 + "\n\n")

            f.write(f"总文件数: {len(processing_results)}\n")
            f.write(f"成功处理: {len(successful)}\n")
            f.write(f"处理失败: {len(failed)}\n\n")

            if successful:
                f.write("成功处理的文件:\n")
                f.write("-" * 30 + "\n")
                total_junctions = 0
                for result in successful:
                    junction_count = result.get('junction_count', 0)
                    total_junctions += junction_count
                    f.write(f"{result['filename']}: {junction_count} 个分叉点\n")

                f.write(f"\n总分叉点数: {total_junctions}\n")
                if successful:
                    f.write(f"平均每张图像: {total_junctions / len(successful):.1f} 个分叉点\n\n")

            if failed:
                f.write("处理失败的文件:\n")
                f.write("-" * 30 + "\n")
                for result in failed:
                    f.write(f"{result['filename']}: {result['error_message']}\n")

        self.logger.info(f"处理报告已保存到: {report_path}")

    def batch_process(self, input_folder_path):
        """批量处理，直接指定文件夹地址"""

        input_folder = Path(input_folder_path)
        if not input_folder.exists() or not input_folder.is_dir():
            self.logger.error(f"输入文件夹不存在或不是文件夹: {input_folder}")
            return

        image_files = self.get_image_files(input_folder)
        if not image_files:
            self.logger.warning("未找到支持的图像文件")
            return

        output_folders = self.create_output_structure(input_folder)

        self.logger.info(f"开始批量处理 {len(image_files)} 个文件")

        processing_results = []
        for i, image_file in enumerate(image_files, 1):
            self.logger.info(f"处理进度: {i}/{len(image_files)} - {image_file.name}")
            result = self.process_single_image(image_file, output_folders)
            processing_results.append(result)

        self.generate_report(processing_results, output_folders['reports'])

        self.logger.info("批量处理完成")


# 主程序直接调用示例，修改路径为你的图片文件夹路径即可
def main():
    input_folder = r"C:\Users\chenb\Desktop\try"  # 这里写死你的目录路径
    processor = NetworkImageProcessor()
    processor.batch_process(input_folder)

if __name__ == "__main__":
    main()


2025-11-22 13:53:32,179 - INFO - 找到 1 个图像文件
2025-11-22 13:53:32,187 - INFO - 开始批量处理 1 个文件
2025-11-22 13:53:32,189 - INFO - 处理进度: 1/1 - ImageCollection_0000046718_2021-02-04 09_06_07 IIIa-19_02_Hematoxylin_Channel.png
C:\Users\chenb\AppData\Local\Temp\ipykernel_21796\3309690912.py:166: UserWarning: Glyph 21407 (\N{CJK UNIFIED IDEOGRAPH-539F}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
C:\Users\chenb\AppData\Local\Temp\ipykernel_21796\3309690912.py:166: UserWarning: Glyph 22987 (\N{CJK UNIFIED IDEOGRAPH-59CB}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
C:\Users\chenb\AppData\Local\Temp\ipykernel_21796\3309690912.py:166: UserWarning: Glyph 22270 (\N{CJK UNIFIED IDEOGRAPH-56FE}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
C:\Users\chenb\AppData\Local\Temp\ipykernel_21796\3309690912.py:166: UserWarning: Glyph 20687 (\N{CJK UNIFIED IDEOGRAPH-50CF}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
C:\Users\chenb\AppData\Local\Temp\ipykernel_21796\33096